## 모델 학습 

* 모델 학습 / 모델 검증 / 예측 등을 다룸 

    <텐서플로 2.0 공식 가이드에서 모델 학습에 권장하는 방법 두 가지></br>
        1) 케라스 모델의 내장 API를 활용하는 방법
        2) 학습, 검증, 예측 모든 과정을 GradientTape 객체를 활용해 직접 구현하는 방법
        
            (1) 장점 : 메서드가 이미 구현 되어있어 간편하다
            (2) 장점 : 좀 더 복잡한 로직을 유연하고 자유롭게 구현할 수 있다.
            (2) 단점 : 일일이 구현해야 함

### 내장 API를 활용하는 방법

* 이미 정의된 케라스의 모델 객체가 있다고 가정해보자
* 이 모델 객체는 케라스의 모델 객체이므로 여러 메서드가 이미 내장되어 있음
* 따라서 내장 메서드를 간단히 사용하기만 하면 됨


</br>

    (1)  학습과정을 정의하기
            - 학습 과정에서 사용 될 손실함수 (loss function), 옵티마이저 (optimizer), 평가에 사용 될 지표 (metric) 등 정의
            

In [ ]:
# 1번 타입

import tensorflow as tf

model.compile(optimizer = tf.keras.optimizer.Adam(),
             loss = tf.keras.losses.CategoricalCrossentropy(),
             metrics = [tf.keras.metrics.Accuracy()])

In [ ]:
# 2번 타입

model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

* 이제 정의된 모델 객체를 대상으로 
    - 학습
    - 평가
    - 예측 메서드를 호출하면 학습이 진행됨
    
    - 아래의 코드와 같이 fit 메서드를 호출하면 데이터들이 모델을 통과하며 학습이 진행된다.
    - 또한, 학습이 진행되며 각 에폭당 모델의 성능(손실 함수, 정확도) 등이 출력됨을 볼 수 있다.

In [ ]:
model.fit(x_train,y_train,
         batch_size = 64,
         epochs =3)

* 학습 과정에서 에폭마다 검증을 진행하는 것 또한 가능하다. 
* evaluate 메서드를 통해 검증 할 수 있지만 매번 에폭을 호출 해야 한다는 번거로움이 있다.
* 따라서 에폭마다 검증 결과를 보기 위해 fit 함수에 검증 데이터를 추가로 넣으면 된다.

In [ ]:
model.fit(x_train, y_train,
         batch_size = 64,
         epochs =3,
         validation_data=(x_val,y_val))

### 더미 데이터를 활용한 감정 분석 모델링

* 이번에 구현할 모델은 심층 신경망(Deep Neural Network) 구조를 사용한다
* 텍스트의 긍정/부정을 예측하는 감정 분석(Sentiment Analysis) 모델
</br></br>
    - 각 단어로 구성된 입력값은 임베딩된 벡터로 변형된다.
    - 이후 각 벡터를 평균해서 하나의 벡터로 만든다.
    - 이후 하나의 은닉층을 거친 후 하나의 결괏값을 뽑는 구조이다.
    - 마지막으로 나온 결괏값에 시그모이드 함수를 적용해 0과 1 사이의 값을 구한다.

### Sequential 로 구현

In [9]:
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow import keras

layers = keras.layers

samples = ['너 오늘 이뻐 보인다',
          '나는 오늘 기분이 더러워',
          '끝내주는데, 좋은 일이 있나봐',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']

labels = [[1],[0],[1],[1],[0],[1]]

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
input_sequences = tokenizer.texts_to_sequences(samples)

word_index = tokenizer.word_index

## 전처리 과정 끝
## 모델 구축, 모델 학습에 필요한 변수 정의


batch_size = 2     
num_epochs = 100
vocab_size =len(word_index)+1
emb_size = 128
hidden_dimension = 256
output_dimension = 1

# Sequential API를 활용해 심층 신경망 모델을 생성할 것임


model = tf.keras.Sequential()
model.add(layers.Embedding(vocab_size, emb_size, input_length = 4))
model.add(layers.Lambda(lambda x: tf.reduce_mean(x, axis = 1)))
model.add(layers.Dense(hidden_dimension, activation ='relu'))
model.add(layers.Dense(output_dimension, activation = 'sigmoid'))

# 입력값을 임베딩하는 Embedding 층을 모델에 추가했다 
# 임베딩 된 각 단어의 벡터를 평균하기 위해 람다(Lamda)층을 사용한다
# lamda층은 텐서플로 연산을 Sequential API 와 Functional API에 적용하기 위해 사용하는 방법임
# 평균의 경우 , 하나의 층으로 정의돼 있지 않기 때문에 람다 층을 활용, 해당 층에 들어오는 입력값들을 평균함
# 평균을 낸 후 은닉층을 통과한 후 최종 출력값을 뽑기 위해 두개의 Dense 층을 모델에 추가

# 최종 출력값을 뽑은 Dense 층과 경우 0과 1 사이의 확률값을 뽑기위해 시그모이드 함수 정의


model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

# 모델을 컴파일하는 메서드를 통해 학습과정을 정의
# 옵티마이저 : Adam 최적화 알고리즘 사용
# 이진 교차 엔트로피 손실함수 (binary cross-entropy) : 이진 분류 문제이므로
# accuracy : 가장 널리 사용되는 정확도

model.fit(input_sequences, labels, epochs=num_epochs, batch_size = batch_size)

2023-03-26 13:18:52.791759: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6929 - accuracy: 0.5000
Epoch 2/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6738 - accuracy: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6597 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6422 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6239 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6029 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 1ms/step - loss: 0.5783 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 1ms/step - loss: 0.5497 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 1ms/step - loss: 0.5164 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 1ms/step - loss: 0.4798 - accuracy: 1.0000
Epoch 11/100
3/3 [===========

3/3 [==============================] - 0s 2ms/step - loss: 7.3675e-04 - accuracy: 1.0000
Epoch 85/100
3/3 [==============================] - 0s 2ms/step - loss: 7.1699e-04 - accuracy: 1.0000
Epoch 86/100
3/3 [==============================] - 0s 2ms/step - loss: 6.9876e-04 - accuracy: 1.0000
Epoch 87/100
3/3 [==============================] - 0s 1ms/step - loss: 6.8238e-04 - accuracy: 1.0000
Epoch 88/100
3/3 [==============================] - 0s 1ms/step - loss: 6.6638e-04 - accuracy: 1.0000
Epoch 89/100
3/3 [==============================] - 0s 1ms/step - loss: 6.4971e-04 - accuracy: 1.0000
Epoch 90/100
3/3 [==============================] - 0s 1ms/step - loss: 6.3502e-04 - accuracy: 1.0000
Epoch 91/100
3/3 [==============================] - 0s 1ms/step - loss: 6.1830e-04 - accuracy: 1.0000
Epoch 92/100
3/3 [==============================] - 0s 1ms/step - loss: 6.0471e-04 - accuracy: 1.0000
Epoch 93/100
3/3 [==============================] - 0s 2ms/step - loss: 5.9044e-04 - accuracy: 

### Functional API 로 구현

In [12]:
import tensorflow as tf
from tensorflow.keras import preprocessing

layers = keras.layers

samples = ['너 오늘 이뻐 보인다',
          '나는 오늘 기분이 더러워',
          '끝내주는데, 좋은 일이 있나봐',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']

labels = [[1],[0],[1],[1],[0],[1]]

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
input_sequences = tokenizer.texts_to_sequences(samples)

word_index = tokenizer.word_index



emb_size = 128
vocab_size = len(word_index)+1



inputs = layers.Input(shape=(4,))
embed_output = layers.Embedding(vocab_size, emb_size)(inputs)
pooled_output = tf.reduce_mean(embed_output, axis = 1)
hidden_layer = layers.Dense(hidden_dimension, activation = 'relu')(pooled_output)
outputs = layers.Dense(output_dimension, activation ='sigmoid')(hidden_layer)
model = tf.keras.Model(inputs = inputs, outputs = outputs)

model.compile(optimizer =tf.keras.optimizers.Adam(0.001),
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])
model.fit(input_sequences, labels, epochs=num_epochs, batch_size = batch_size)

Epoch 1/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6899 - accuracy: 0.8333
Epoch 2/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6714 - accuracy: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6545 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6375 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6156 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 1ms/step - loss: 0.5942 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 1ms/step - loss: 0.5635 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 1ms/step - loss: 0.5288 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 1ms/step - loss: 0.4900 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 1ms/step - loss: 0.4470 - accuracy: 1.0000
Epoch 11/100
3/3 [===========

3/3 [==============================] - 0s 1ms/step - loss: 7.3704e-04 - accuracy: 1.0000
Epoch 85/100
3/3 [==============================] - 0s 1ms/step - loss: 7.1907e-04 - accuracy: 1.0000
Epoch 86/100
3/3 [==============================] - 0s 1ms/step - loss: 7.0008e-04 - accuracy: 1.0000
Epoch 87/100
3/3 [==============================] - 0s 1ms/step - loss: 6.8518e-04 - accuracy: 1.0000
Epoch 88/100
3/3 [==============================] - 0s 1ms/step - loss: 6.6744e-04 - accuracy: 1.0000
Epoch 89/100
3/3 [==============================] - 0s 2ms/step - loss: 6.5083e-04 - accuracy: 1.0000
Epoch 90/100
3/3 [==============================] - 0s 2ms/step - loss: 6.3588e-04 - accuracy: 1.0000
Epoch 91/100
3/3 [==============================] - 0s 2ms/step - loss: 6.2097e-04 - accuracy: 1.0000
Epoch 92/100
3/3 [==============================] - 0s 1ms/step - loss: 6.0688e-04 - accuracy: 1.0000
Epoch 93/100
3/3 [==============================] - 0s 1ms/step - loss: 5.9415e-04 - accuracy: 

### Subclassing 방법으로 구현

In [15]:
import tensorflow as tf
from tensorflow.keras import preprocessing

layers = keras.layers

samples = ['너 오늘 이뻐 보인다',
          '나는 오늘 기분이 더러워',
          '끝내주는데, 좋은 일이 있나봐',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']

labels = [[1],[0],[1],[1],[0],[1]]

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
input_sequences = tokenizer.texts_to_sequences(samples)

word_index = tokenizer.word_index





emb_size = 128
vocab_size = len(word_index)+1

class CustomModel(tf.keras.Model):
    
    def __init__(self, vocab_size, embed_dimension, hidden_dimension, output_dimension):
        super(CustomModel, self).__init__(name='my_model')
        self.embedding = layers.Embedding(vocab_size, embed_dimension)
        self.dense_layer = layers.Dense(hidden_dimension, activation ='relu')
        self.output_layer = layers.Dense(output_dimension, activation = 'sigmoid')
        
    def call(self, inputs):
        x = self.embedding(inputs)
        x = tf.reduce_mean(x, axis=1)
        x = self.dense_layer(x)
        x = self.output_layer(x)
        
        
        return x
    
    
model = CustomModel(vocab_size = vocab_size,
                   embed_dimension=emb_size,
                   hidden_dimension = hidden_dimension,
                   output_dimension = output_dimension)

model.compile(optimizer = tf.keras.optimizers.Adam(0.001),
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])

model.fit(input_sequences, labels, epochs=num_epochs, batch_size = batch_size)

Epoch 1/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6969 - accuracy: 0.3333
Epoch 2/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6780 - accuracy: 0.8333
Epoch 3/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6631 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 1ms/step - loss: 0.6473 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6309 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6133 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5923 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5666 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5377 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5037 - accuracy: 1.0000
Epoch 11/100
3/3 [===========

3/3 [==============================] - 0s 2ms/step - loss: 7.8944e-04 - accuracy: 1.0000
Epoch 85/100
3/3 [==============================] - 0s 2ms/step - loss: 7.6925e-04 - accuracy: 1.0000
Epoch 86/100
3/3 [==============================] - 0s 1ms/step - loss: 7.4950e-04 - accuracy: 1.0000
Epoch 87/100
3/3 [==============================] - 0s 2ms/step - loss: 7.3139e-04 - accuracy: 1.0000
Epoch 88/100
3/3 [==============================] - 0s 2ms/step - loss: 7.1364e-04 - accuracy: 1.0000
Epoch 89/100
3/3 [==============================] - 0s 2ms/step - loss: 6.9710e-04 - accuracy: 1.0000
Epoch 90/100
3/3 [==============================] - 0s 2ms/step - loss: 6.8054e-04 - accuracy: 1.0000
Epoch 91/100
3/3 [==============================] - 0s 1ms/step - loss: 6.6676e-04 - accuracy: 1.0000
Epoch 92/100
3/3 [==============================] - 0s 2ms/step - loss: 6.5005e-04 - accuracy: 1.0000
Epoch 93/100
3/3 [==============================] - 0s 2ms/step - loss: 6.3658e-04 - accuracy: 